In [ ]:
#| default_exp prepare_images

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

# "Process images" bit of bedmap pipeline

## bedmap overview

bedmap does three things:
- processes images through to embeddings
- creates layouts from embeddings
- builds web assets

Design goals:
- terse (<500 LOC)
- performant (10k images in 10 minutes)
- readable

## process_images

This notebook does "process image" part (first bullet above):
1. Validates images
2. Creates thumbnails
3. Generates embeddings


In [ ]:
#| export

from pathlib import Path
import shutil
import sys

import daft
from humanize import naturalsize
from tqdm import tqdm
from loguru import logger

from bedmap.validate_images import validate_images
from bedmap.create_thumbnails import create_thumbnails
from bedmap.embed_images import embed_images
from bedmap.config import ImageConfig

from fastcore.test import test_eq

{'image_dir': None, 'image_glob': None, 'thumbnail_size': 128, 'model_name': 'timm/vit_small_patch14_reg4_dinov2.lvd142m', 'umap_config': {'umap_n_components': 10, 'umap_n_neighbors': 15, 'umap_min_dist': 0.1}}


In [ ]:
#| export

# easy timestamps

logger.remove()
logger.add(sys.stdout, level="INFO")

1

In [ ]:
#| export

cfg = ImageConfig()

BATCH_SIZE = 4
MODEL_NAME = cfg.model_name

## 0. Create dataframe with images

In [ ]:
#| export

def _image_glob_pattern(directory: str | Path) -> str:
    """Convert a directory path into a glob pattern that matches common image formats.
    
    Args:
        directory: Directory path to search for images
        
    Returns:
        Glob pattern string that matches common image formats
    """
    exts = ['jpg', 'jpeg', 'png', 'gif', 'bmp', 'webp', 'tiff', 'tif']
    dir_str = str(directory).rstrip('/')
    return f"{dir_str}/**/*.{{{','.join(exts)}}}"
    

In [ ]:
#| hide

test_eq(_image_glob_pattern("my/dir"),
        "my/dir/**/*.{jpg,jpeg,png,gif,bmp,webp,tiff,tif}")

In [ ]:
#| export

def _df_images_from_pattern(pattern: str | Path) -> daft.DataFrame:
    """Create a dataframe from a glob pattern of images.
    
    Args:
        pattern: Directory or glob pattern to match image files
        
    Returns:
        DataFrame with columns:
        - path: Path to the file
        - size: Size in bytes
    """
    pattern = str(pattern)
    if '*' not in pattern: pattern = _image_glob_pattern(pattern)
    df_img = daft.from_glob_path(pattern).with_column_renamed("path", "img_path")
    df_img = df_img.with_column("img_name", df_img["img_path"].str.split("/").list.get(-1).cast(str))
    df_img = df_img.with_column(
        "img", daft.col("img_path").url.download(on_error="null").image.decode(on_error="null", mode="RGB")
    ).exclude("num_rows")
    return df_img

In [ ]:
#| hide

test_dir = "../tests/test-data/smithsonian_butterflies_10/jpgs"
images_df = _df_images_from_pattern(test_dir)
images_df.show(1)

╭────────────────────────────────┬────────┬────────────────────────────────┬────────────╮
│ img_path                       ┆ size   ┆ img_name                       ┆ img        │
│ ---                            ┆ ---    ┆ ---                            ┆ ---        │
│ Utf8                           ┆ Int64  ┆ Utf8                           ┆ Image[RGB] │
╞════════════════════════════════╪════════╪════════════════════════════════╪════════════╡
│ file://../tests/test-data/smi… ┆ 149480 ┆ 30aeb051d-ee0d-4c5b-8a85-a8da… ┆ <Image>    │
╰────────────────────────────────┴────────┴────────────────────────────────┴────────────╯

(Showing first 1 rows)

In [ ]:
#| export

def dl_hf_images(dataset_name: str = "kvriza8/microscopy_images",
                 dir: Path = None,
                 max_images: int = 64,
                 overwrite: bool = True,
                 format: str = "png") -> None:
    """Download images from a Hugging Face dataset.
    
    Args:
        dataset_name: Name of the Hugging Face dataset to download from
        dir: Directory to save images to. If None, creates a directory named after the dataset
        max_images: Maximum number of images to download
        overwrite: Whether to overwrite existing directory if it exists
        format: Image format to save as (png, jpg, etc)
        
    Returns:
        None
    """
    from datasets import load_dataset

    dataset = load_dataset(dataset_name, split="train", streaming=True)
    if overwrite:
        shutil.rmtree(dir, ignore_errors=True)
        dir.mkdir(parents=True, exist_ok=True)

    image_paths = []
    for i, img_row in enumerate(tqdm(iter(dataset), total=max_images)):
        if i >= max_images:
            break
        img = img_row["image"]
        image_paths += [(dir / f"{i}.{format}")]
        img.save(image_paths[-1])

    print(f"Size of images on disk: {naturalsize(sum([p.stat().st_size for p in image_paths]))}")

    return None

## 1. Validate images

In [ ]:
#| hide

images_df, invalid_df = validate_images(images_df)

Checking size_nontrivial
Checking img_not_oblong


🗡️ 🐟 Project: 00:00 

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 Filter: 00:00 

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 Aggregate: 00:00 

🗡️ 🐟 Project: 00:00 

Checking img_name_distinct


## 2. Generate thumbnails

In [ ]:
#| hide

images_df = create_thumbnails(images_df, height=128)
images_df.show(2)

╭────────────────────────────────┬────────┬────────────────────────────────┬────────────┬────────────╮
│ img_path                       ┆ size   ┆ img_name                       ┆ img        ┆ thumb      │
│ ---                            ┆ ---    ┆ ---                            ┆ ---        ┆ ---        │
│ Utf8                           ┆ Int64  ┆ Utf8                           ┆ Image[RGB] ┆ Image[RGB] │
╞════════════════════════════════╪════════╪════════════════════════════════╪════════════╪════════════╡
│ file://../tests/test-data/smi… ┆ 149480 ┆ 30aeb051d-ee0d-4c5b-8a85-a8da… ┆ <Image>    ┆ <Image>    │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ file://../tests/test-data/smi… ┆ 82911  ┆ 9fea3150-a3d4-11ed-aeea-e36f1… ┆ <Image>    ┆ <Image>    │
╰────────────────────────────────┴────────┴────────────────────────────────┴────────────┴────────────╯

(Showing first 2 rows)

## 3. Generate embeddings

In [ ]:
#| export

def _embed_images_for_df(df: daft.DataFrame) -> daft.DataFrame:
    """
    Embed images for a given dataframe.
    """
    ## Surely there's a cleaner way to get the paths out

    paths = [Path(i["img_path"].lstrip("file:/")) for i in df.select("img_path").to_pylist()]
    embeds = embed_images(paths, model_name=MODEL_NAME, batch_size=BATCH_SIZE)
    embeds_type = daft.DataType.embedding(daft.DataType.float32(), embeds.shape[-1])
    embeds_series = daft.Series.from_numpy(embeds).cast(embeds_type)

    return images_df.with_column("embeds", daft.lit(embeds_series))

In [ ]:
#| hide

images_df = _embed_images_for_df(images_df)

images_df.show(1)


╭────────────────────────────┬────────┬────────────────────────────┬────────────┬────────────┬─────────────────────────╮
│ img_path                   ┆ size   ┆ img_name                   ┆ img        ┆ thumb      ┆ embeds                  │
│ ---                        ┆ ---    ┆ ---                        ┆ ---        ┆ ---        ┆ ---                     │
│ Utf8                       ┆ Int64  ┆ Utf8                       ┆ Image[RGB] ┆ Image[RGB] ┆ Embedding[Float32; 384] │
╞════════════════════════════╪════════╪════════════════════════════╪════════════╪════════════╪═════════════════════════╡
│ file://../tests/test-data/ ┆ 149480 ┆ 30aeb051d-ee0d-4c5b-8a85-a ┆ <Image>    ┆ <Image>    ┆ <Embedding>             │
│ smi…                       ┆        ┆ 8da…                       ┆            ┆            ┆                         │
╰────────────────────────────┴────────┴────────────────────────────┴────────────┴────────────┴─────────────────────────╯

(Showing first 1 rows)

In [ ]:
# | hide

import nbdev

nbdev.nbdev_export()

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/nbdev/doclinks.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources,importlib
